# Get all recipe
from https://veenaazmanov.com/recipes-in-text

In [1]:
import requests
from bs4 import BeautifulSoup

main_page = requests.get('https://veenaazmanov.com/recipes-in-text')
html_main = main_page.content

soup = BeautifulSoup(html_main, 'html.parser')

all_recipe = soup.find_all('a', target='_self')
print(f'Tags with self targeted found: {len(all_recipe)}')

Tags with self targeted found: 1362


In [2]:
all_recipe[:2]

[<a data-wpel-link="internal" href="https://veenaazmanov.com/" rel="follow noopener noreferrer" target="_self">Veena Azmanov</a>,
 <a data-wpel-link="internal" href="https://veenaazmanov.com/cook/" rel="follow noopener noreferrer" target="_self"><span>Cook</span></a>]

We can see that some of them don't refer to recipe pages. From observation, first 49 tag of them don't.

In [3]:
all_recipe = all_recipe[49:]   # First 49 urls are not recipe

all_recipe_url = [a.get('href') for a in all_recipe]   # Grab only url from href attribute of each tag
menus = [a.string for a in all_recipe]   # Grab the menu name

In [4]:
all_recipe_url[:3]

['https://veenaazmanov.com/homemade-chocolate-birthday-cake-recipe/',
 'https://veenaazmanov.com/chocolate-fudge-cake/',
 'https://veenaazmanov.com/gluten-free-chocolate-cake-recipe/']

In [31]:
with open('unique_url.txt','w') as f:
    f.write('\n'.join(unique_url))

In [5]:
import sys
import time

data = {}
non_menu_url = []
error_url = []
unique_url = list(set(all_recipe_url))  # To make sure there is no duplicated url

for url in unique_url:

    try:
        page = requests.get(url).content
    except requests.exceptions.MissingSchema as e:
        error_url.append(url)
        continue

    soup = BeautifulSoup(page, 'html.parser')
    
    try:
        menu = soup.find("h2", class_="wprm-recipe-name wprm-block-text-bold")
        if menu is None : raise AttributeError
    except AttributeError as e:
        non_menu_url.append(url)
        continue

    steps = [a for a in soup.find_all("div", class_="wprm-recipe-instruction-text")]
    data[str(menu.string)] = steps

    sys.stdout.write("\rData revieved: %i" % len(data))
    sys.stdout.flush()
    time.sleep(0.05)

Data revieved: 10

In [6]:
print(f"Data collected: {len(data)}")
print(f"Non recipe url detected: {len(non_menu_url)}")
print(f"Failed in get request: {len(error_url)}")

Data collected: 10
Non recipe url detected: 0
Failed in get request: 0


In [19]:
data_strings_new = {
    key: [str(step.text) for step in steps if step.text != None] for key, steps in data.items()
}

In [20]:
import json
import os

if not os.path.exists('data_strings_local.json'):
    json.dump(data_strings_new, open('data_strings_local.json', 'w'))

In [21]:
test_load = json.load(open('data_strings_local.json','r'))
print(f'Nuber of recipe: {len(test_load.keys())}')

Nuber of recipe: 719


In [34]:
test_load['Crock-Pot Beef Stew']

['Add carrots, potatoes, beef, onion, garlic, ginger, herbs, and spices into the slow cooker.',
 'In a bowl or measuring cup – combine broth, red wine, Worcestershire sauce, vinegar, tomato paste, and mustard.',
 'Then, pour the liquid ingredients over the slow cooker ingredients.',
 'Cover and cook on low-heat for 6 hours or on high for 3 hours.',
 'Combine cornstarch and water in a bowl. Then, add it to the slow cooker.',
 'Continue to cook on high for another hour.',
 'Serve over steamed rice, couscous, or cauliflower rice.']